# CSCI 4253 / 5253 - Lab #4 - Patent Problem with Spark DataFrames
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

This [Spark cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf) is useful as is [this reference on doing joins in Spark dataframe](http://www.learnbymarketing.com/1100/pyspark-joins-by-example/).

The [DataBricks company has one of the better reference manuals for PySpark](https://docs.databricks.com/spark/latest/dataframes-datasets/index.html) -- they show you how to perform numerous common data operations such as joins, aggregation operations following `groupBy` and the like.

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

The following aggregation functions may be useful -- [these can be used to aggregate results of `groupby` operations](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html#example-aggregations-using-agg-and-countdistinct). More documentation is at the [PySpark SQL Functions manual](https://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html#module-pyspark.sql.functions). Feel free to use other functions from that library.

In [2]:
from pyspark.sql.functions import col, count, countDistinct
import pyspark.sql.functions

Create our session as described in the tutorials

In [3]:
spark = SparkSession \
    .builder \
    .appName("Lab4-Dataframe") \
    .master("local[*]")\
    .getOrCreate()

Read in the citations and patents data and check that the data makes sense. Note that unlike in the RDD solution, the data is automatically inferred to be Integer() types.

In [4]:
citations = spark.read.load('cite75_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [5]:
citations.show(5)

+-------+-------+
| CITING|  CITED|
+-------+-------+
|3858241| 956203|
|3858241|1324234|
|3858241|3398406|
|3858241|3557384|
|3858241|3634889|
+-------+-------+
only showing top 5 rows



In [6]:
patents = spark.read.load('apat63_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [7]:
patents.show(5)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
|3070801| 1963| 1096|   null|     BE|   null|    null|      1|  null|   269|  6|    69| null|       1|    null|    0.0|    null|    null|    null|    null|    null|    null|    null|
|3070802| 1963| 1096|   null|     US|     TX|    null|      1|  null|     2|  6|    63| null|       0|    null|   null|    null|    null|    null|    null|    null|    null|    null|
|3070803| 1963| 1096|   null|     US|     IL|    null|      1|  null|     2|  6|    6

Modified the Patent Table, by selecting only the Patent and the State Columns, as the others are unnecessary.

In [8]:
ModifiedPatentsTable = patents.select('PATENT','POSTATE')
ModifiedPatentsTable.show()

+-------+-------+
| PATENT|POSTATE|
+-------+-------+
|3070801|   null|
|3070802|     TX|
|3070803|     IL|
|3070804|     OH|
|3070805|     CA|
|3070806|     PA|
|3070807|     OH|
|3070808|     IA|
|3070809|     AZ|
|3070810|     IL|
|3070811|     CA|
|3070812|     LA|
|3070813|     NY|
|3070814|     MN|
|3070815|     CO|
|3070816|     OK|
|3070817|     RI|
|3070818|     IN|
|3070819|     TN|
|3070820|   null|
+-------+-------+
only showing top 20 rows



Implemented an Inner Table join between ModifiedPatentsTable and citations; based on the CITED number in citations, with PATENT number in ModifiedPatentsTable

In [9]:
FirstTableJoin = citations.join(ModifiedPatentsTable,citations.CITED == ModifiedPatentsTable.PATENT, 'inner')
FirstTableJoin.show()

+-------+-------+-------+-------+
| CITING|  CITED| PATENT|POSTATE|
+-------+-------+-------+-------+
|3951073|3071083|3071083|   null|
|4401034|3071083|3071083|   null|
|4802416|3071083|3071083|   null|
|4045189|3071452|3071452|   null|
|4192656|3071452|3071452|   null|
|3949375|3071753|3071753|     MN|
|4120573|3071753|3071753|     MN|
|4271479|3071753|3071753|     MN|
|4345315|3071753|3071753|     MN|
|4496943|3071753|3071753|     MN|
|5062619|3071941|3071941|   null|
|3861359|3072100|3072100|     IL|
|3894516|3072100|3072100|     IL|
|4138968|3072100|3072100|     IL|
|4280448|3072100|3072100|     IL|
|4572109|3072100|3072100|     IL|
|3917094|3072274|3072274|     WI|
|4396343|3072274|3072274|     WI|
|4907934|3072274|3072274|     WI|
|4955781|3072274|3072274|     WI|
+-------+-------+-------+-------+
only showing top 20 rows



Selected just the required columns from the above table join - CITING, CITED and CITED_STATE. The last column of POSTATE has been aliased to be CITED_STATE, as that makes more sense.

In [10]:
firstOutput = FirstTableJoin.select('CITING','CITED',col('POSTATE').alias("CITED_STATE"))
firstOutput.show()

+-------+-------+-----------+
| CITING|  CITED|CITED_STATE|
+-------+-------+-----------+
|3951073|3071083|       null|
|4401034|3071083|       null|
|4802416|3071083|       null|
|4045189|3071452|       null|
|4192656|3071452|       null|
|3949375|3071753|         MN|
|4120573|3071753|         MN|
|4271479|3071753|         MN|
|4345315|3071753|         MN|
|4496943|3071753|         MN|
|5062619|3071941|       null|
|3861359|3072100|         IL|
|3894516|3072100|         IL|
|4138968|3072100|         IL|
|4280448|3072100|         IL|
|4572109|3072100|         IL|
|3917094|3072274|         WI|
|4396343|3072274|         WI|
|4907934|3072274|         WI|
|4955781|3072274|         WI|
+-------+-------+-----------+
only showing top 20 rows



Created a second inner join between the ModifiedPatentsTable and the above table. The join is based on the CITING number present in the above table and the PATENT number of the ModifiedPatentsTable. Through this join, I was able to obtain the CITING_STATE.

In [11]:
SecondTableJoin = firstOutput.join(ModifiedPatentsTable,firstOutput.CITING == ModifiedPatentsTable.PATENT, 'inner')
SecondTableJoin.show()

+-------+-------+-----------+-------+-------+
| CITING|  CITED|CITED_STATE| PATENT|POSTATE|
+-------+-------+-----------+-------+-------+
|3858527|3638586|         CA|3858527|   null|
|3858527|3699902|         OH|3858527|   null|
|3858527|3602157|         TX|3858527|   null|
|3858597|3815160|         NY|3858597|     MT|
|3858597|3675252|         AZ|3858597|     MT|
|3859029|3741706|         OH|3859029|     NY|
|3859029|3685936|         OH|3859029|     NY|
|3859627|3368197|         MI|3859627|     MI|
|3859627|3226674|       null|3859627|     MI|
|3859627|3359539|         CA|3859627|     MI|
|3859627|3729059|         NY|3859627|     MI|
|3859627|3381268|         DC|3859627|     MI|
|3859627|3462732|       null|3859627|     MI|
|3859627|3340523|         FL|3859627|     MI|
|3860038|3430989|         CA|3860038|     AZ|
|3860038|3359351|         TX|3860038|     AZ|
|3860038|3105570|         MA|3860038|     AZ|
|3860100|3797623|         OH|3860100|     IL|
|3860100|3182776|         IN|38601

Obtained the Second required output, which will be of the form - CITING, CITING_STATE, CITED, CITED_STATE. This is obtained by dropping the PATENT column from the above table; and then selecting the other required columns. Also, the column POSTATE is aliased again to be CITING_STATE.

In [12]:
secondOutput = SecondTableJoin.drop('PATENT').select('CITING',col('POSTATE').alias('CITING_STATE'),'CITED','CITED_STATE')
secondOutput.show()

+-------+------------+-------+-----------+
| CITING|CITING_STATE|  CITED|CITED_STATE|
+-------+------------+-------+-----------+
|3858527|        null|3638586|         CA|
|3858527|        null|3699902|         OH|
|3858527|        null|3602157|         TX|
|3858597|          MT|3815160|         NY|
|3858597|          MT|3675252|         AZ|
|3859029|          NY|3741706|         OH|
|3859029|          NY|3685936|         OH|
|3859627|          MI|3368197|         MI|
|3859627|          MI|3226674|       null|
|3859627|          MI|3359539|         CA|
|3859627|          MI|3729059|         NY|
|3859627|          MI|3381268|         DC|
|3859627|          MI|3462732|       null|
|3859627|          MI|3340523|         FL|
|3860038|          AZ|3430989|         CA|
|3860038|          AZ|3359351|         TX|
|3860038|          AZ|3105570|         MA|
|3860100|          IL|3797623|         OH|
|3860100|          IL|3182776|         IN|
|3860100|          IL|3638773|         OH|
+-------+--

Obtained the third required output, wherein only the same state citation rows are obtained from the above table. This will help to get the count of the same state citations in the next stage.

In [13]:
thirdOutput = secondOutput.where(secondOutput.CITING_STATE == secondOutput.CITED_STATE)
thirdOutput.show()

+-------+------------+-------+-----------+
| CITING|CITING_STATE|  CITED|CITED_STATE|
+-------+------------+-------+-----------+
|4178878|          AK|3464385|         AK|
|3974004|          AL|3745074|         AL|
|3974004|          AL|3585090|         AL|
|3974004|          AL|3762972|         AL|
|3974004|          AL|3692600|         AL|
|4554823|          AL|3373564|         AL|
|4698246|          AL|3972467|         AL|
|4701360|          AL|3972467|         AL|
|5078406|          AL|5026073|         AL|
|5701722|          AL|5623808|         AL|
|5661858|          AR|5309583|         AR|
|3881518|          AZ|3589601|         AZ|
|4323097|          AZ|4192362|         AZ|
|4653557|          AZ|3994326|         AZ|
|4656646|          AZ|4445094|         AZ|
|4656646|          AZ|4438413|         AZ|
|4656646|          AZ|4601048|         AZ|
|4656646|          AZ|4359692|         AZ|
|4724471|          AZ|3777216|         AZ|
|4757494|          AZ|4466092|         AZ|
+-------+--

Here, the count of same state citations is obtained from the above table. This is done by first grouping based on the CITING number column and then aggregating the counts if CITING_STATE is same as CITED_STATE in the above table. To obtain the count, I have used the pyspark.sql.functions.count() function.

In [14]:
citingSameStateCount = thirdOutput.groupby('CITING').agg(pyspark.sql.functions.count(thirdOutput.CITING_STATE == thirdOutput.CITED_STATE).alias('SAME_STATE_COUNT'))
citingSameStateCount.show()

+-------+----------------+
| CITING|SAME_STATE_COUNT|
+-------+----------------+
|4240165|               3|
|5096364|               2|
|5122917|               5|
|5203482|               2|
|5583013|               9|
|5409826|               2|
|4053654|               1|
|4829378|               1|
|5393360|               2|
|4661390|               2|
|5179032|               2|
|5638851|               2|
|5602785|               2|
|4237332|               4|
|5405045|               1|
|4255566|               1|
|4312201|               2|
|5522077|               5|
|5999308|               3|
|5954264|               1|
+-------+----------------+
only showing top 20 rows



The final output is obtained by doing an Inner join between the original patents table and the above table which contains the same state counts. This is done by comparing the PATENT column in the original patents table with the CITING column in the above table.

In [15]:
finalOutput = patents.join(citingSameStateCount, patents.PATENT == citingSameStateCount.CITING, 'inner')
finalOutput.show()

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-------+----------------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD| CITING|SAME_STATE_COUNT|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-------+----------------+
|3859627| 1975| 5485|   1972|     US|     MI|  332120|      2|    14|   180|  5|    55|    7|       2|     1.0|    0.5|  0.4082|    12.5|  6.8571|     0.0|     0.0|     0.0|     0.0|3859627|               1|
|3860191| 1975| 5492|   1972|     US|     CA|  452885|      2|     5|   242|  5|    51|    4|       4|    0.75|    0.0|  0.4444|     5.0|    11.0|     0.0|     0.0|    

This is to obtain the Final Required Output, which is to show the top 10 results when the count of the same-state citations is in the descending order.

In [16]:
finalRequiredOutput = finalOutput.orderBy('SAME_STATE_COUNT',ascending=False).show(10)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-------+----------------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD| CITING|SAME_STATE_COUNT|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-------+----------------+
|5959466| 1999|14515|   1997|     US|     CA|    5310|      2|  null|   326|  4|    46|  159|       0|     1.0|   null|  0.6186|    null|  4.8868|  0.0455|   0.044|    null|    null|5959466|             125|
|5983822| 1999|14564|   1998|     US|     TX|  569900|      2|  null|   114|  5|    55|  200|       0|   0.995|   null|  0.7201|    null|   12.45|     0.0|     0.0|    